In [1]:
# RSA mathematics foundation and simple implementing
# 
#
# 质数/素数
#   只能被1和自身整除
#
# 互质
#   最大公约数为1的数
#
# 欧拉函数
#   欧拉函数φ(n)是小于或等于正整数n的数字中与n互质的数的数目
#   如果n是两个质数p和q的乘积, 那么:
#   φ(n) = φ(p*q) = φ(p) * φ(q) = (p - 1) * (q - 1)
#
# 模反元素
#   如果两个正整数a和n互质，那么一定可以找到整数b，使得 a*b - 1 被n整除，或者说a*b被n除的余数是1
#   a*b ≡ 1 (mod n)  
#
# 欧拉定理
#   如果两个正整数a和n互质，则n的欧拉函数 φ(n) 可以让下面的等式成立：
#   a^φ(n) ≡ 1 (mod n)
#   a * a^(φ(n) - 1) ≡ 1 (mod n)
#   由此可得：a的φ(n) - 1次方肯定是a关于n的模反元素
#   欧拉定理证明模反元素必然存在
#
# 费马小定理
#   假设正整数a与质数p互质，因为质数p的φ(p)等于p-1，则欧拉定理可以写成
#   a^(p-1) ≡ 1 (mod p)
#   费马小定理是欧拉定理的一个特例
#

In [2]:
# RSA encrypt and decrypt, and prove
#   m: message, m_e: encrypted message, m_e_d: decrypted message from m_e
# 
#   m_e ≡ m^e (mod n)
#   m_e_d ≡ m_e^d (mod n) ≡ m^(e*d) (mod n)
#   e*d ≡ 1 (mod φ(n)) -> e*d = hφ(n) + 1
#   m_e_d ≡ m^(hφ(n) + 1) (mod n) ≡ m * m^(hφ(n)) ≡ m * (m^φ(n))^h ≡ m * (1)^h (mod n) ≡ m
# 

In [3]:
# RSA generator steps
# step1: choose two different primes
#   p, q
#   n = p * q  : modulus
#
# step2: get r
#   r = φ(n) = φ(pq) = φ(p) * φ(q) = (p - 1) * (q - 1)
#
# step3: get e
#   1 < e < r
#   e, r coprime
#
# step4: get d
#   d: 1 < d < r and d * e mod r == 1
#   d = (h * r + 1) / e
#
# step5: 
#   (e, n) : public key
#   (d, n) : private key
#

In [17]:
%%latex
\begin{equation}
欧拉函数: \phi(n) = \phi(pq) = \phi(p) \times \phi(q) = (p - 1) \times (q -1) \\
r = \phi(n), 1 < e < r, 1 < d < r \\
模反元素: e \times d \equiv 1 \pmod{r}, e \times d \equiv 1 \pmod{\phi(n)} \\
\\
欧拉定理: m^{\phi(n)} \equiv 1 \pmod{n} \\
\textit{欧拉定理可以证明模反元素必然存在}: m^{\phi(n)} = m \times m^{\phi(n) - 1} \equiv 1 \pmod{n} \\
\\
m: message \space (m < n)\\
加密: m_e = m^{e} \bmod{n} \\
解密: m_{e\_d} = m_e^{d} \bmod{n} \\
证明: m_e^{d} \bmod{n} = m^{e \times d} \bmod{n} = m^{h\phi(n) + 1} \bmod{n} = m \times m^{h\phi(n)} \bmod{n} = m \times 1 = m
\\                                
在线时长 = \sum_{i=1}^N(logtime_{i+1} - logtime_i), 如果 logtime_{i+1} - logtime_i > 30 分钟, 则记为1分钟
\end{equation}

<IPython.core.display.Latex object>

In [30]:
IGNORE_FIRST_N_PRIMES = 3
P_AND_Q_MIN_DIFF = 2
E_AND_D_MIN_DIFF = 1

In [31]:
def primes_generator():
    primes = []   # primes generated so far
    last = 1      # last number tried
    while True:
        last += 1
        for p in primes:
            if last % p == 0:
                break
        else:
            primes.append(last)
            yield last

# greatest common divisor
def gcd(x, y):
   while(y):
       x, y = y, x % y
        
   return x

# least common multiple
def lcm(x, y):
   return (x * y) // gcd(x, y)

In [32]:
def get_p_q():
    primes = primes_generator()
    for i in range(1, IGNORE_FIRST_N_PRIMES):
        next(primes)   
    p = next(primes)

    for i in range(1, P_AND_Q_MIN_DIFF):
        next(primes)
    q = next(primes)
    
    return (p, q)
        
def get_n(p, q):
    return p * q


def get_r(p, q):
    #return lcm(p - 1, q - 1)
    return (p -1) * (q - 1)


def get_e(r):
    for e in range(2, r):
        if gcd(e, r) == 1:
            return e
        else:
            continue
            
    return 0


def get_d(r, e):
    for i in range(1, 10000000):
        x = divmod(i * r + 1, e)
        quotient = x[0]
        remainder = x[1]
        if remainder > 0:
            continue
        
        if quotient > r:
            break
        
        if quotient > 1 and quotient != e and abs(quotient - e) > E_AND_D_MIN_DIFF:
            return quotient
        else:
            continue
            
    return 0    

In [33]:
def get_rsa(p, q):
    rsa = dict()
    rsa['p'] = p
    rsa['q'] = q
    rsa['n'] = p * q
    
    rsa['r'] = get_r(p, q)
    r = rsa['r']
    
    e = get_e(r)
    rsa['e'] = e
    if e == 0:
        rsa['ready'] = False
        return rsa
    
    d = get_d(r, e)
    rsa['d'] = d
    if d == 0:
        rsa['ready'] = False
        return rsa

    rsa['ready'] = True
    return rsa

In [34]:
def verify_rsa(rsa, msgs):
    for e, d in (('e', 'd'), ('d', 'e')):
        print("test: m -> {}(m) -> {}({}(m)) ...".format(e, d, e))
        print("--------------------------------")
        for m in msgs:
            m_e = m ** rsa[e] % rsa['n']
            m_e_d = m_e ** rsa[d] % rsa['n']
            print("{status}: {m:4d} -> {m_e:4d} -> {m_e_d:4d}".format(
                status=('PASS' if m == m_e_d else 'FAIL'), 
                m=m, 
                m_e=m_e, 
                m_e_d=m_e_d)
            )
        print("")

In [35]:
def main():
    p, q = get_p_q()
    rsa = get_rsa(p, q)
    print('RSA -> {}'.format(rsa))
    if rsa['ready'] == False:
        print('Get RSA failed!')
        return
    
    CASE_COUNT = 10
    print('RSA verify:')
    print("=====================================================")
    print("m < rsa['n'] cases:")
    verify_rsa(rsa, range(max(rsa['n'] - CASE_COUNT, 1), rsa['n']))
    print("=====================================================")
    print("m >= rsa['n'] cases:")
    verify_rsa(rsa, range(rsa['n'], rsa['n'] + CASE_COUNT))

In [36]:
main()

RSA -> {'p': 5, 'q': 11, 'n': 55, 'r': 40, 'e': 3, 'd': 27, 'ready': True}
RSA verify:
m < rsa['n'] cases:
test: m -> e(m) -> d(e(m)) ...
--------------------------------
PASS:   45 ->   45 ->   45
PASS:   46 ->   41 ->   46
PASS:   47 ->   38 ->   47
PASS:   48 ->   42 ->   48
PASS:   49 ->    4 ->   49
PASS:   50 ->   40 ->   50
PASS:   51 ->   46 ->   51
PASS:   52 ->   28 ->   52
PASS:   53 ->   47 ->   53
PASS:   54 ->   54 ->   54

test: m -> d(m) -> e(d(m)) ...
--------------------------------
PASS:   45 ->   45 ->   45
PASS:   46 ->   51 ->   46
PASS:   47 ->   53 ->   47
PASS:   48 ->   27 ->   48
PASS:   49 ->   14 ->   49
PASS:   50 ->   30 ->   50
PASS:   51 ->    6 ->   51
PASS:   52 ->   13 ->   52
PASS:   53 ->   37 ->   53
PASS:   54 ->   54 ->   54

m >= rsa['n'] cases:
test: m -> e(m) -> d(e(m)) ...
--------------------------------
FAIL:   55 ->    0 ->    0
FAIL:   56 ->    1 ->    1
FAIL:   57 ->    8 ->    2
FAIL:   58 ->   27 ->    3
FAIL:   59 ->    9 ->    4
FAI

In [ ]:
# The ASN.1 spec for the PKCS1 RSA private key format is as follows:
#
# RSAPrivateKey ::= SEQUENCE {
#     version           Version,
#     modulus           INTEGER,  -- n
#     publicExponent    INTEGER,  -- e
#     privateExponent   INTEGER,  -- d
#     prime1            INTEGER,  -- p
#     prime2            INTEGER,  -- q
#     exponent1         INTEGER,  -- d mod (p-1)
#     exponent2         INTEGER,  -- d mod (q-1)
#     coefficient       INTEGER,  -- (inverse of q) mod p
#     otherPrimeInfos   OtherPrimeInfos OPTIONAL
# }
#

In [1]:
%%latex
\begin{equation}
\textit{coefficient = (inverse of q) mod p} \\
\textit{when p is prime} \\
coefficient = q^{p-2} \bmod{p}
\end{equation}

<IPython.core.display.Latex object>

In [ ]:
# reference:
#
# https://en.wikipedia.org/wiki/X.509
# https://zh.wikipedia.org/wiki/RSA%E5%8A%A0%E5%AF%86%E6%BC%94%E7%AE%97%E6%B3%95
# https://en.wikipedia.org/wiki/RSA_(cryptosystem)
# https://zhuanlan.zhihu.com/p/33580225
# https://tools.ietf.org/html/rfc3447
# https://tools.ietf.org/html/rfc5280
# https://tools.ietf.org/html/rfc5280
# https://zhuanlan.zhihu.com/p/36326221
# https://andrea.corbellini.name/2015/05/17/elliptic-curve-cryptography-a-gentle-introduction/
#